In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 6

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 14

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
3286,"[740.8939819335938, 1501.4183349609375]","[741.1802978515625, 1502.5081787109375]","[740.83642578125, 1503.7806396484375]","[740.8794555664062, 1504.842529296875]","[740.8195190429688, 1505.9658203125]","[740.7850952148438, 1507.1068115234375]","[740.7318115234375, 1508.2723388671875]","[740.6721801757812, 1509.448974609375]","[740.6036376953125, 1510.633056640625]","[740.5287475585938, 1511.8206787109375]",...,"[739.759521484375, 1525.0880126953125]","[739.706787109375, 1526.3592529296875]","[739.6576538085938, 1527.6611328125]","[739.6144409179688, 1529.001953125]","[739.5789794921875, 1530.387451171875]","[739.550048828125, 1531.81591796875]","[739.5206298828125, 1533.2728271484375]","[739.4788818359375, 1534.7318115234375]","[739.4136352539062, 1536.163818359375]","[739.3206176757812, 1537.5517578125]"
11220,"[1824.134033203125, 438.99737548828125]","[1823.0977783203125, 438.26806640625]","[1821.95068359375, 437.43896484375]","[1820.9251708984375, 436.5596008300781]","[1819.9254150390625, 435.7091064453125]","[1818.9322509765625, 434.8560791015625]","[1817.9454345703125, 434.00360107421875]","[1816.96875, 433.1564025878906]","[1816.003173828125, 432.318603515625]","[1815.0479736328125, 431.4917297363281]",...,"[1804.539306640625, 422.5767517089844]","[1803.5621337890625, 421.7564392089844]","[1802.585693359375, 420.937255859375]","[1801.6123046875, 420.1210021972656]","[1800.6441650390625, 419.3096923828125]","[1799.683837890625, 418.50537109375]","[1798.7337646484375, 417.7101135253906]","[1797.7962646484375, 416.9256896972656]","[1796.8734130859375, 416.1536560058594]","[1795.9671630859375, 415.3949890136719]"
39229,"[709.6897583007812, 1143.0673828125]","[709.3652954101562, 1142.2882080078125]","[708.9292602539062, 1141.5238037109375]","[708.76513671875, 1140.8157958984375]","[708.576171875, 1140.07470703125]","[708.359375, 1139.321533203125]","[708.1294555664062, 1138.54443359375]","[707.8870239257812, 1137.741943359375]","[707.6345825195312, 1136.9189453125]","[707.37646484375, 1136.08251953125]",...,"[705.2921142578125, 1127.4107666015625]","[705.2089233398438, 1126.658203125]","[705.1407470703125, 1125.8980712890625]","[705.0864868164062, 1125.1278076171875]","[705.0451049804688, 1124.3455810546875]","[705.0158081054688, 1123.550048828125]","[704.998046875, 1122.7403564453125]","[704.9912109375, 1121.916259765625]","[704.9949340820312, 1121.077880859375]","[705.0089111328125, 1120.2254638671875]"
24311,"[2066.137451171875, 625.52880859375]","[2067.439697265625, 626.2571411132812]","[2068.784912109375, 627.225341796875]","[2070.089111328125, 628.1848754882812]","[2071.42919921875, 629.1627197265625]","[2072.77587890625, 630.1412963867188]","[2074.127685546875, 631.1260375976562]","[2075.481689453125, 632.1221923828125]","[2076.83837890625, 633.1387939453125]","[2078.199462890625, 634.185546875]",...,"[2093.088134765625, 647.9825439453125]","[2094.513916015625, 649.5186767578125]","[2096.018310546875, 651.2015991210938]","[2097.44775390625, 652.8196411132812]","[2098.6748046875, 654.1853637695312]","[2099.673828125, 655.251220703125]","[2100.482177734375, 656.06201171875]","[2101.169189453125, 656.7024536132812]","[2101.8798828125, 657.3284301757812]","[2103.015869140625, 658.340087890625]"
5657,"[239.81045532226562, 1025.415771484375]","[240.26730346679688, 1025.3447265625]","[240.73126220703125, 1025.3721923828125]","[241.08303833007812, 1025.3873291015625]","[241.49745178222656, 1025.4356689453125]","[241.9065399169922, 1025.475830078125]","[242.32810974121094, 1025.5203857421875]","[242.753662109375, 1025.562744140625]","[243.1830596923828, 1025.6026611328125]","[243.61390686035156, 1025.6380615234375]",...,"[248.20004272460938, 1025.7535400390625]","[248.60093688964844, 1025.7642822265625]","[249.00042724609375, 1025.780029296875]","[249.39865112304688, 1025.8011474609375]","[249.79563903808594, 1025.828125]","[250.19131469726562, 1025.8612060546875]","

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
3286,740.893982,1501.418335,741.180298,1502.508179,740.836426,1503.780640,740.879456,1504.842529,740.819519,1505.965820,...,739.550049,1531.815918,739.520630,1533.272827,739.478882,1534.731812,739.413635,1536.163818,739.320618,1537.551758
11220,1824.134033,438.997375,1823.097778,438.268066,1821.950684,437.438965,1820.925171,436.559601,1819.925415,435.709106,...,1799.683838,418.505371,1798.733765,417.710114,1797.796265,416.925690,1796.873413,416.153656,1795.967163,415.394989
39229,709.689758,1143.067383,709.365295,1142.288208,708.929260,1141.523804,708.765137,1140.815796,708.576172,1140.074707,...,705.015808,1123.550049,704.998047,1122.740356,704.991211,1121.916260,704.994934,1121.077881,705.008911,1120.225464
24311,2066.137451,625.528809,2067.439697,626.257141,2068.784912,627.225342,2070.089111,628.184875,2071.429199,629.162720,...,2099.673828,655.251221,2100.482178,656.062012,2101.169189,656.702454,2101.879883,657.328430,2103.015869,658.340088
5657,239.810455,1025.415771,240.267303,1025.344727,240.731262,1025.372192,241.083038,1025.387329,241.497452,1025.435669,...,250.191315,1025.861206,250.585510,1025.900146,250.977997,1025.945190,251.368530,1025.995972,251.756897,1026.052612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33393,1915.015503,507.442596,1915.939209,508.247437,1916.864868,508.960236,1917.707520,509.707367,1918.618774,510.495544,...,1937.374023,527.039734,1938.340576,527.954651,1939.323730,528.894531,1940.309937,529.839417,1941.281006,530.760986
30179,172.239136,1024.366821,172.875671,1024.516968,173.676086,1024.586426,174.435471,1024.652954,175.267090,1024.768799,...,192.092117,1025.806396,192.862946,1025.912842,193.627960,1026.024414,194.380585,1026.134521,195.114197,1026.237061
18801,601.252625,1386.664185,600.883057,1386.624634,600.491638,1386.640869,600.086548,1386.616333,599.733276,1386.633789,...,592.883850,1386.542480,592.612793,1386.534058,592.333740,1386.520142,592.043335,1386.499146,591.738586,1386.469360


In [12]:
df2.to_csv("outputs14epb6.csv", index=True)